# Massachusetts Senior Circuit Breaker Credit

The Massachusetts Senior Circuit Breaker Credit provides a refundable tax credit to seniors, depending on their age, income, household structure, rent, property tax, and property value.

## Examples

Consider households with one or two adults aged 65, paying \$1,000 monthly rent and receiving \$1,000 monthly Social Security income per person.
The Senior Circuit Breaker Credit provides up to \$189 to the single person and up to \$1,170 to the couple.
It spikes up at \$15,200 for a single person because they become disqualified from SNAP and lose the generous minimum benefit (thanks to SNAP emergency allotments), and the SCB treats SNAP as income.

In [1]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_scb(adults):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=65, rent=12_000, social_security=12_000)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse", age=65, social_security=12_000)
        members += ["spouse"]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MA")
    sim.vary("employment_income", max=20_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            ma_senior_circuit_breaker=sim.calc("ma_senior_circuit_breaker")[
                0
            ].round(),
            scb_mtr=-sim.deriv(
                "ma_senior_circuit_breaker",
                "employment_income",
                wrt_target="head",
            ),
            adults=adults,
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    l.append(make_scb(adults))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    ma_senior_circuit_breaker="MA Senior Circuit Breaker Credit",
    scb_mtr="Senior Circuit Breaker Credit Marginal Tax Rate",
    adults="Adults",
)

fig = px.line(
    df,
    "employment_income",
    "ma_senior_circuit_breaker",
    color="adults",
    labels=LABELS,
    title="Massachusetts Senior Circuit Breaker Credit",
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

The credit phases out at 10% of income where it phases out smoothly.

In [2]:
fig = px.line(
    df,
    "employment_income",
    "scb_mtr",
    color="adults",
    labels=LABELS,
    title="Massachusetts Senior Circuit Breaker Credit marginal tax rate",
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".1%",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
    yaxis_range=[0, 0.2],
)
fig.show()